# Conociendo un poco el comportamiento del Ozono ($0_3$)

In [ ]:
import pandas as pd
import altair as alt

In [ ]:
# Cargar los datos
data = pd.read_csv(
    'http://www.aire.cdmx.gob.mx/opendata/anuales_horarios_gz/contaminantes_2021.csv.gz', 
    skiprows=10
)

In [ ]:
data.head(15)

In [ ]:
d = data['date']
pd.to_datetime(d, format='%d/%m/%Y %H:%M')

In [ ]:
def fix_24hr(dt):
    dt_fix = dt[:-5] + '00' + dt[-3:]
    return dt_fix

In [ ]:
data['date'] = pd.to_datetime(
    d.where(~d.str.contains('24:00'), lambda x: x.apply(fix_24hr)), 
    format='%d/%m/%Y %H:%M')

In [ ]:
data.head(15)

In [ ]:
chart = alt.Chart(data[
                       (data['id_parameter'] == 'O3') & 
                       (data['date'].dt.month == 3) &
                       (data['id_station'] == 'PED')
                       ])

# Scipy

Una biblioteca que extiende las funcionalidades de **Numpy** en cosas de cálculo numérico en algunos temas:

* Funciones especiales
* Integración
* Optimización
* Estadística
* Estructuras de datos
* Manejo de imágenes


## Balanceo de ecuaciones 

En un soplete se quema acetileno ($\textrm{C}_2\textrm{H}_2$) entonces la reacción que se lleva a cabo es:
$$
    \textrm{C}_2\textrm{H}_2 + \textrm{O}_2 \to 
        \textrm{CO}_2 + \textrm{H}_2\textrm{O}
$$
¿Cómo podemos balancear esta ecuación?

In [ ]:
from scipy import linalg

## Modelando las temperaturas de Alaska en el año

Tenemos las temperaturas extremas, máximas y mínimas, de cada mes en Alaska.

Creemos que podemos modelar la temperatura a lo largo del año usando la formula

$$
    T(t) = \alpha + \omega\cos\left(\frac{2\pi(t+t_{\textrm{off}})}{12}\right)
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

temp_max = np.array([17,  19,  21,  28,  33,  38, 37,  37,  31,  23,  19,  18])
temp_min = np.array([-62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58])

In [ ]:
from scipy import optimize

Además se tiene las funciones:

* `optimize.root`
* `optimize.minimize`

## Observando como crece una población

Verhulst en el siglo XIX propuso un modelo para ver la dinámica del crecimiento poblacional. El modelo lo podemos escribir de la siguiente manera

$$
\frac{\textrm{d}P}{\textrm{d}t} = r P \left(1-\frac{P}{K}\right) 
$$

In [ ]:
from scipy import integrate


Algunas otras funciones interesantes:

* `integrate.quad`
* `integrate.dblquad`
* `integrate.tplquad`
* `integrate.trapezoid`